In [11]:
import pandas as pd;
import numpy as np;
import plotly.express as px
artists=pd.read_csv("data/artists.csv")


# Artist Dataset


In [12]:
artists.sample(5)

,id,followers,genres,name,popularity
788240,1wNEnOmQX8MQhfvdFi5QPV,110.0,['zhongguo feng'],國風堂,25
345392,5n926jGLQbg2NlYCUrZaxQ,25.0,[],The House Theatre of Chicago,0
134171,3C0q9u0OMlQDImOFmPrQZB,0.0,[],Quinto Maganini,0
290597,1mhwOkskTFBQQ6BtRUaVUM,164.0,['mluvene slovo'],Ladislav Smoljak,0
979654,4P4Bev03M6bPkZxJmScuti,33.0,[],Aslane,4


# The most popular genre

In [13]:
artists["TmpGenres"]=artists["genres"].apply(lambda x: x.strip('][').split(','))#convert arrays
allGeners=np.unique(np.concatenate(artists["TmpGenres"]))
allGeners[0]="unknown"



In [14]:
import operator
macroGeneres = dict.fromkeys(
    ["alternative", "rock", "metal", "pop", "rap", "punk", "jazz", "reggae", "soul", "polka", "country", "electronic",
     "funk", "hip hop", "r&b", "folk", "house", "techno", "trance", "indie", "blues", "instrument", "electronic",
     "disco", "deep", "hardcore", "wave","trap" ,"other"], 0)
totalGener={}


def getMacroCategory(str):
    counts = dict.fromkeys(macroGeneres.keys(),0)
    for e in str:
        words = e
        for x in macroGeneres.keys():
            if words.count(x)>0:
              counts[x]+=1
    counts = {key: value for key, value in sorted(counts.items(), key=lambda item: (-item[1],item[0]), reverse=False)}
    #print(counts)
    if len(counts)>0:
        if list(counts.keys())[0] in list(macroGeneres.keys()):
            macroGeneres[list(counts.keys())[0]] += 1
            return list(counts.keys())[0]
        else:
            return "other"
    return "other"

artists["macro genre"]=artists.apply(lambda z:getMacroCategory(z["TmpGenres"]),axis=1)


In [15]:
generesDf={'Generes':macroGeneres.keys(),'Number':macroGeneres.values()}

macroGeneresDF=pd.DataFrame.from_dict(generesDf)
macroGeneresDF=macroGeneresDF.sort_values(by="Number",ascending=False)


In [16]:
fig=px.histogram(macroGeneresDF,y="Generes",x="Number")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["xaxis"]["title"]="Generes"
fig.layout["yaxis"]["title"]="Number of Artists"
fig.layout.yaxis.dtick=0.5
fig.show()

## Relationship between followers and popularity

In [17]:
tmpData=artists.sample(10000);
fig=px.scatter(tmpData,x="followers",y="popularity",hover_name=tmpData["name"], log_x=True)#
fig.show()

## The top 20 of the most popular metal artists

KeyError: False

In [21]:
metalArtists=artists.loc[artists["macro genre"]=="metal"]
top20Metal=metalArtists.sort_values(by="popularity",ascending=False)
fig=px.histogram(top20Metal[:20],x="popularity",y="name")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["yaxis"]["title"]="Band"
fig.layout["xaxis"]["title"]="Popolarity"
fig.show()

# Maximum Popularity per macro-genre

In [29]:
maxPerGenere={}

for a in macroGeneres:
    tmp=artists.loc[artists["macro genre"]==a]
    if(np.size(tmp)>0):
        maxPerGenere[a]=tmp.sort_values(by="popularity",ascending=False).iloc[0]["popularity"]




In [30]:

fig=px.histogram(pd.DataFrame.from_dict({'Genere':maxPerGenere.keys(),"Popularity":maxPerGenere.values()}).sort_values(by="Popularity",ascending=False)[1:],x="Genere",y="Popularity")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["yaxis"]["title"]="Popularity"
fig.layout["xaxis"]["title"]="Genre"
fig.show()